In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
# Load JSON
with open('/content/combined_train.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract Urdu and English sentences
urdu_sentences = [item['translation']['ur'] for item in data]
english_sentences = ['<sos> ' + item['translation']['en'] + ' <eos>' for item in data]


In [3]:
# Urdu tokenizer
urdu_tokenizer = Tokenizer()
urdu_tokenizer.fit_on_texts(urdu_sentences)
urdu_sequences = urdu_tokenizer.texts_to_sequences(urdu_sentences)

# English tokenizer
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)

# Get vocab sizes
urdu_vocab_size = len(urdu_tokenizer.word_index) + 1
eng_vocab_size = len(eng_tokenizer.word_index) + 1

# Padding
max_urdu_len = max(len(seq) for seq in urdu_sequences)
max_eng_len = max(len(seq) for seq in eng_sequences)

encoder_input = pad_sequences(urdu_sequences, maxlen=max_urdu_len, padding='post')
decoder_input = pad_sequences([seq[:-1] for seq in eng_sequences], maxlen=max_eng_len-1, padding='post')
decoder_target = pad_sequences([seq[1:] for seq in eng_sequences], maxlen=max_eng_len-1, padding='post')


In [4]:
embedding_dim = 256
latent_dim = 512

# Encoder
encoder_inputs = Input(shape=(max_urdu_len,))
x = Embedding(urdu_vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(x)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_eng_len-1,))
x = Embedding(eng_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(x, initial_state=encoder_states)
decoder_dense = Dense(eng_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 517)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 239)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 517, 256)  │ 19,888,384 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 239, 256)  │  9,478,656 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 512),     │  1,574,912 │ embedding[0][0]   │
│                     │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 239,      │  1,574,912 │ embedding_1[0][0… │
│                     │ 512), (None,      │            │ lstm[0][1],       │
│                     │ 512), (None,      │            │ lstm[0][2]        │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 239,       │ 18,994,338 │ lstm_1[0][0]      │
│                     │ 37026)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 51,511,202 (196.50 MB)

 Trainable params: 51,511,202 (196.50 MB)

 Non-trainable params: 0 (0.00 B)

###LSTM

In [6]:
model.fit([encoder_input, decoder_input],
          np.expand_dims(decoder_target, -1),
          batch_size=64,
          epochs=5,
          validation_split=0.2)


Epoch 1/5
575/575 ━━━━━━━━━━━━━━━━━━━━ 756s 1s/step - accuracy: 0.8902 - loss: 1.4279 - val_accuracy: 0.8768 - val_loss: 0.8907
Epoch 2/5
575/575 ━━━━━━━━━━━━━━━━━━━━ 829s 1s/step - accuracy: 0.8985 - loss: 0.7196 - val_accuracy: 0.8816 - val_loss: 0.8311
Epoch 3/5
575/575 ━━━━━━━━━━━━━━━━━━━━ 801s 1s/step - accuracy: 0.9029 - loss: 0.6611 - val_accuracy: 0.8842 - val_loss: 0.7998
Epoch 4/5
575/575 ━━━━━━━━━━━━━━━━━━━━ 789s 1s/step - accuracy: 0.9069 - loss: 0.6108 - val_accuracy: 0.8863 - val_loss: 0.7804
Epoch 5/5
575/575 ━━━━━━━━━━━━━━━━━━━━ 820s 1s/step - accuracy: 0.9102 - loss: 0.5670 - val_accuracy: 0.8874 - val_loss: 0.7717


###RNN

In [14]:
from tensorflow.keras.layers import SimpleRNN

# Encoder
encoder_inputs_rnn = Input(shape=(max_urdu_len,))
x = Embedding(urdu_vocab_size, embedding_dim)(encoder_inputs_rnn)
encoder_outputs, state_h = SimpleRNN(latent_dim, return_state=True)(x)
encoder_states_rnn = [state_h]

# Decoder
decoder_inputs_rnn = Input(shape=(max_eng_len-1,))
x = Embedding(eng_vocab_size, embedding_dim)(decoder_inputs_rnn)
decoder_rnn = SimpleRNN(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_rnn(x, initial_state=encoder_states_rnn)
decoder_dense_rnn = Dense(eng_vocab_size, activation='softmax')
decoder_outputs = decoder_dense_rnn(decoder_outputs)

# Model
rnn_model = Model([encoder_inputs_rnn, decoder_inputs_rnn], decoder_outputs)
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
rnn_model.summary()

# Training
rnn_model.fit([encoder_input, decoder_input],
              np.expand_dims(decoder_target, -1),
              batch_size=64,
              epochs=5,
              validation_split=0.2)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 517)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 239)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 517, 256)  │ 19,888,384 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 239, 256)  │  9,478,656 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_3        │ [(None, 512),     │    393,728 │ embedding_5[0][0] │
│ (SimpleRNN)         │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_4        │ [(None, 239,      │    393,728 │ embedding_6[0][0… │
│ (SimpleRNN)         │ 512), (None,      │            │ simple_rnn_3[0][… │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 239,       │ 18,994,338 │ simple_rnn_4[0][… │
│                     │ 37026)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 49,148,834 (187.49 MB)

 Trainable params: 49,148,834 (187.49 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-pac

### GRU

In [15]:
from tensorflow.keras.layers import GRU

# Encoder
encoder_inputs_gru = Input(shape=(max_urdu_len,))
x = Embedding(urdu_vocab_size, embedding_dim)(encoder_inputs_gru)
encoder_outputs, state_h = GRU(latent_dim, return_state=True)(x)
encoder_states_gru = [state_h]

# Decoder
decoder_inputs_gru = Input(shape=(max_eng_len-1,))
x = Embedding(eng_vocab_size, embedding_dim)(decoder_inputs_gru)
decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(x, initial_state=encoder_states_gru)
decoder_dense_gru = Dense(eng_vocab_size, activation='softmax')
decoder_outputs = decoder_dense_gru(decoder_outputs)

# Model
gru_model = Model([encoder_inputs_gru, decoder_inputs_gru], decoder_outputs)
gru_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
gru_model.summary()

# Training
gru_model.fit([encoder_input, decoder_input],
              np.expand_dims(decoder_target, -1),
              batch_size=64,
              epochs=5,
              validation_split=0.2)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 517)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, 239)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_7         │ (None, 517, 256)  │ 19,888,384 │ input_layer_7[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_8         │ (None, 239, 256)  │  9,478,656 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ [(None, 512),     │  1,182,720 │ embedding_7[0][0] │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ [(None, 239,      │  1,182,720 │ embedding_8[0][0… │
│                     │ 512), (None,      │            │ gru[0][1]         │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 239,       │ 18,994,338 │ gru_1[0][0]       │
│                     │ 37026)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 50,726,818 (193.51 MB)

 Trainable params: 50,726,818 (193.51 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5


OperatorNotAllowedInGraphError: Exception encountered when calling GRU.call().

[1mIterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.[0m

Arguments received by GRU.call():
  • sequences=tf.Tensor(shape=(64, 517, 256), dtype=float32)
  • initial_state=None
  • mask=None
  • training=True